In [1]:
from nnfs.datasets import spiral_data
import matplotlib.pyplot as plt
import numpy as np
import nnfs
nnfs.init()

In [2]:
X, y = spiral_data(samples=100, classes=3)

In [3]:
class Layer_Dense:
    def __init__(self, n_inputs, n_neurons):
        # initialize weights and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
    
    # forward pass
    def forward(self, inputs):
        self.output = np.dot(inputs, self.weights) + self.biases

In [4]:
# ReLU activation
class Activation_ReLU:
    # Forward pass
    def forward ( self , inputs ):
        # Calculate output values from inputs
        self.output = np.maximum(0, inputs)

class Activation_Softmax:

    # forward pass
    def forward(self, inputs):
        # handle overflowing error
        inputs = [np.subtract(input, np.max(input)) for input in inputs]
        # calculate exponential values
        ex_values = np.exp(inputs)
        sum_ex_values = [np.sum(ex_value) for ex_value in ex_values] # option2: np.sum(ex_values, axis=1)
        self.output = [ex_value / sum_ex_value for ex_value, sum_ex_value in zip(ex_values, sum_ex_values)]

In [7]:
# Loss function - categorical cross-entropy
class Loss_CategoricalCrossentropy:
    def forward(self, y_pred, y_true):
        # Number of samples in a batch
        samples = len(y_pred)

        # Clip data to prevent division by zero and negative log
        y_pred_clipped = np.clip(y_pred, 1e-7, 1 - 1e-7)
        
        # Probabilities for target values if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true] # >> [0.7 0.8 0.7]
            
        # Mask values for one-hot encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1) # >> [0.7 0.8 0.7]
        
        # Calculate the negative log likelihoods
        negative_log_likelihoods = -np.log(correct_confidences) # >> [0.35667494 0.22314355 0.35667494]
        
        # Return the mean loss
        self.output =  np.mean(negative_log_likelihoods)

In [9]:
dense_layer_1 = Layer_Dense(2, 3)
activation_layer_1 = Activation_ReLU()
dense_layer_2 = Layer_Dense(3, 3)
activation_layer_2 = Activation_Softmax()

loss_function = Loss_CategoricalCrossentropy()

dense_layer_1.forward(X)
activation_layer_1.forward(dense_layer_1.output)
dense_layer_2.forward(activation_layer_1.output)
activation_layer_2.forward(dense_layer_2.output)
loss_function.forward(activation_layer_2.output, y)

print(loss_function.output)

1.0986179
